In [1]:
import cv2
import numpy as np

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from glob import glob

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.applications import MobileNetV2

In [3]:
global image_h
global image_w
global num_landmarks

""" Hyperparameters """
image_h = 512
image_w = 512
num_landmarks = 106
input_shape = (image_h, image_w, 3)
batch_size = 32
lr = 1e-3
num_epochs = 100

In [4]:
""" Paths """
path = "./LaPa"

In [5]:
train_img = sorted(glob(os.path.join(path, "train", "images", "*.jpg")))
train_land = sorted(glob(os.path.join(path, "train", "landmarks", "*.txt")))
valid_img = sorted(glob(os.path.join(path, "val", "images", "*.jpg")))
valid_land = sorted(glob(os.path.join(path, "val", "landmarks", "*.txt")))
test_img = sorted(glob(os.path.join(path, "test", "images", "*.jpg")))
test_land = sorted(glob(os.path.join(path, "test", "landmarks", "*.txt")))
# train_img

In [6]:
#train_img = train_img.decode()

In [7]:
def read_image_lankmarks(image_path, landmark_path):
    """ Image """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    h, w, _ = image.shape
    image = cv2.resize(image, (image_w, image_h))
    
    # Cast the image data to float32
    image = tf.cast(image, dtype=tf.float32)
    image /= 255.0
  #  image = image.astype(np.float32)

    """ Lankmarks """
    data = open(landmark_path, "r").read()
    lankmarks = []

    for line in data.strip().split("\n")[1:]:
        x, y = line.split(" ")
        x = float(x)/w
        y = float(y)/h

        lankmarks.append(x)
        lankmarks.append(y)

    lankmarks = np.array(lankmarks, dtype=np.float32)

    return image, lankmarks

In [8]:
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image, landmarks = read_image_lankmarks(x, y)
        return image, landmarks

    image, landmarks = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    image.set_shape([image_h, image_w, 3])
    landmarks.set_shape([num_landmarks * 2])

    return image, landmarks

In [9]:
def build_model(input_shape, num_landmarks):
    inputs = L.Input(input_shape)

    backbone = MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs, alpha=0.5)
    backbone.trainable = True

    x = backbone.output
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.2)(x)
    outputs = L.Dense(num_landmarks*2, activation="sigmoid")(x)

    model = tf.keras.models.Model(inputs, outputs)
    return model

In [10]:
ds_train = tf.data.Dataset.from_tensor_slices((train_img, train_land))
ds_valid = tf.data.Dataset.from_tensor_slices((valid_img, valid_land))

# for i in ds_train:
#     print(i)
#     print(i[0])
#     print(i[1])
#     print('-----------------------------------------------------------------------------------------') 

In [11]:
ds_train = ds_train.shuffle(buffer_size=5000)
ds_valid = ds_valid.shuffle(buffer_size=5000)
ds_train

<_ShuffleDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [12]:
ds_train = ds_train.map(preprocess)
ds_valid = ds_valid.map(preprocess)
ds_train

<_MapDataset element_spec=(TensorSpec(shape=(512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(212,), dtype=tf.float32, name=None))>

In [13]:
batch=8
ds_train = ds_train.batch(batch).prefetch(2)
ds_valid = ds_valid.batch(batch).prefetch(2)
# Most dataset input pipelines should end with a call to prefetch. 
# This allows later elements to be prepared while the current element is being processed. 
# This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
ds_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 212), dtype=tf.float32, name=None))>

In [14]:
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")

In [15]:
""" Model """
model = build_model(input_shape, num_landmarks)
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr))

""" Training """
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

In [16]:
 """ Seeding """
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model.fit(ds_train,
        validation_data=ds_train,
        epochs=num_epochs,
        callbacks=callbacks
    )

Epoch 1/100
   8/2271 [..............................] - ETA: 1:23:44 - loss: 0.6903